# Partie 3 - Modele Predictif 2 : Logistic Regression

## Introduction

PRESENTATION DU MODELE LOGISTIC REGRESSION

## Import des librairies et du dataset

On commence par importer les librairies nécessaires, donc numpy et pandas. Ensuite, on importe le dataset.

In [357]:
import pandas as pd
import numpy as np

data = pd.read_csv('heart_disease_by_ceo.csv', index_col=0)

## Selection des "features" et de la valeur "cible"

On va ensuite selectionner la valeur cible (ici la colonne "HeartDisease") et les features (toutes les colonnes sauf "HeartDisease").


In [358]:
X = data.drop(columns=["HeartDisease"])
Y = data["HeartDisease"]

## Separation des données en données d'entrainement et de test

On va ensuite entrainer le modele Logistic Regression sur les données d'entrainement à l'aide de train_test_split.

On va proceder de la maniere suivante, 80% de données pour l'entrainement et 20% pour le test. Comme pour le modèle KNN.

In [359]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.20, random_state=42)

## Normalisation des données

On va ensuite Normaliser les données d'entrainement et de test à l'aide de StandardScaler afin de les mettre à la même échelle. Cette étape est très importante pour les modèles de Regression Logistique.

In [360]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


## Entrainement du modèle

On va ensuite entrainer le modèle Logistic Regression sur les données d'entrainement à l'aide de la fonction fit(). C'est a partir de cette étape que le modèle change par rapport au modèle KNN.

In [361]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(X_train, Y_train)


LogisticRegression()

## Prédictions

On va ensuite faire des prédictions sur les données de test à l'aide de la fonction predict().

In [362]:
from sklearn.metrics import accuracy_score, classification_report
y_pred = model.predict(X_test)

## Evaluation du modèle

On va enfin évaluer le modèle en utilisant la fonction accuracy_score() pour calculer la précision du modèle et classification_report() pour afficher les résultats.


In [363]:
print(accuracy_score(Y_test, y_pred))
print("\n")
print("---------------------------------------")
print("\n")
print(classification_report(Y_test, y_pred))

0.8695652173913043


---------------------------------------


              precision    recall  f1-score   support

           0       0.80      0.89      0.84        72
           1       0.92      0.86      0.89       112

    accuracy                           0.87       184
   macro avg       0.86      0.87      0.87       184
weighted avg       0.87      0.87      0.87       184



Puis la matrice de confusion pour voir les résultats de manière plus visuelle.

In [364]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(Y_test, y_pred))
print("\n")
print("---------------------------------------")
print("\n")
print(pd.crosstab(Y_test, y_pred, rownames=['Actual'], colnames=['Predicted'], margins=True))

[[64  8]
 [16 96]]


---------------------------------------


Predicted   0    1  All
Actual                 
0          64    8   72
1          16   96  112
All        80  104  184


## Test du modèle

On va maintenant tester le modèle sur un exemple de données pour voir si le modèle prédit correctement la valeur cible.

In [365]:
liste_colonnes = ['Age', 'Sex', 'RestingBP', 'Cholesterol', 'FastingBS', 
                  'MaxHR', 'ExerciseAngina', 'Oldpeak', 'TA_ChestPain', 
                  'ATA_ChestPain', 'NAP_ChestPain', 'ASY_ChestPain', 
                  'Normal_RestingECG', 'ST_RestingECG', 'LVH_RestingECG', 
                  'Up_ST_Slope', 'Flat_ST_Slope', 'Down_ST_Slope']


patient = [[40,0,140,289,0,172,0,0.0,0,1,0,0,1,0,0,1,0,0]] #1
patient2 = [[57,1,130,236,0,174,0,0.0,0,1,0,0,0,0,1,0,1,0]] #916
patient3 = [[57,0,130,131,0,115,1,1.2,0,0,0,1,1,0,0,0,1,0]] #915
patient4 = [[59,0,164,176,1,90,0,1.0,0,0,0,1,0,0,1,0,1,0]] #911

patient_df = pd.DataFrame(patient, columns=liste_colonnes)
patient2_df = pd.DataFrame(patient2, columns=liste_colonnes)
patient3_df = pd.DataFrame(patient3, columns=liste_colonnes)
patient4_df = pd.DataFrame(patient4, columns=liste_colonnes)

patient = scaler.transform(patient_df)
patient2 = scaler.transform(patient2_df)
patient3 = scaler.transform(patient3_df)
patient4 = scaler.transform(patient4_df)

print("Predictions")
print(model.predict(patient))
print(model.predict(patient2))
print(model.predict(patient3))
print(model.predict(patient4))

patients = []
patients.append(patient)
patients.append(patient2)
patients.append(patient3)
patients.append(patient4)

# Je cree un dataframe pour stocker les resultats
resultats = pd.DataFrame(columns=['Sain', 'Malade'])
noms_patients = [f'Patient {i+1}' for i in range(len(patients))]

for i, p in enumerate(noms_patients):
    probs = model.predict_proba(patients[i])
    resultats.loc[p] = probs[0]

print("\nProbabilités de maladie cardiaque par patient:")
print(resultats)

Predictions
[0]
[0]
[1]
[1]

Probabilités de maladie cardiaque par patient:
               Sain    Malade
Patient 1  0.962897  0.037103
Patient 2  0.890541  0.109459
Patient 3  0.043601  0.956399
Patient 4  0.030505  0.969495


### Optimisation du modele

83% d'accuracy, c'est pas mal mais on peut faire mieux. On va essayer d'optimiser le modèle en utilisant GridSearchCV pour trouver les meilleurs hyperparamètres.

Explication de la valeur C : 
C: float, default=1.0 Inverse of regularization strength; must be a positive float. Like in support vector machines, smaller values specify stronger regularization.


On va ensuite entrainer le modèle Logistic Regression sur les données d'entrainement à l'aide de GridSearchCV.

Dans GridSearchCV, on va utiliser la grille de paramètres suivants :

- 'C': [0.1, 1, 10, 100, 1000] qui correspond à l'inverse de la force de régularisation. Plus la valeur de C est petite, plus le modèle est régularisé.

- 'penalty': ['l1', 'l2'] qui correspond à la norme de la régularisation. L1 est la régularisation Lasso et L2 est la régularisation Ridge.

- 'solver': ['liblinear', 'saga'] qui correspond à l'optimiseur utilisé pour résoudre le problème d'optimisation. liblinear est utilisé pour les petites datasets et saga pour les grands datasets.







In [366]:
from sklearn.model_selection import GridSearchCV


param_grid = {
    'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'penalty': ['l1', 'l2'],  
    'solver': ['liblinear','saga']  
}
model = LogisticRegression(max_iter=1000)
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy', n_jobs=-1)

grid_search.fit(X_train, Y_train)
results = pd.DataFrame(grid_search.cv_results_)

print("Best Parameters:", grid_search.best_params_)


best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

accuracy = accuracy_score(Y_test, y_pred)
print(f"Best Model Accuracy: {accuracy:.4f}")
print(classification_report(Y_test, y_pred))


Best Parameters: {'C': 0.1, 'penalty': 'l2', 'solver': 'saga'}
Best Model Accuracy: 0.8641
              precision    recall  f1-score   support

           0       0.79      0.89      0.84        72
           1       0.92      0.85      0.88       112

    accuracy                           0.86       184
   macro avg       0.86      0.87      0.86       184
weighted avg       0.87      0.86      0.87       184

